<img src="escudo_utfsm.gif" style="float:right;height:100px">
<img src="IsotipoDIisocolor.png" style="float:left;height:100px">
<center>
    <h1> ILI286 - Computación Científica II</h1>
    <h1> Tarea 1: Deflation y Matrices Sparse </h1> 
    <h3> Victor Torres Varas 201173076-3</h3>
    <h4> victor.torresva@alumnos.usm.cl</h4>
</center>
<p>
<center>Octubre 2019 - v1.0 </center>
</p>

---

In [1]:
import numpy as np
import scipy as sp
import scipy.sparse as sparse
import time
from scipy import sparse
from time import time

## Contexto

Esta tarea tiene como objetivo aplicar los contenidos vistos en clase, respecto a métodos numéricos para obtener valores y vectores propios. Además, esta instancia servirá para recordar (o aprender) el uso de *Jupyter Notebook*, junto con las principales bibliotecas del curso: *Numpy* y *Scipy*.

## Sección 1: Algoritmo Deflation (30 puntos)

Se discutió en clases que una de las principales propiedades que tienen las matrices simétricas es que

* Sus valores propios son reales.
* Sus vectores propios son ortogonales.

Sabemos que por medio de *Power Iteration* e *Inverse Power Iteration*, podemos rescatar algunos de los valores propios y vectores propios. Mediante *Normalized Simultaneous Iteration* o *Unshifted QR*, podemos extraer todos los valores y vectores propios al mismo tiempo. Ahora veremos un método distinto para extraer los valores y vectores propios, uno por uno.

Consideremos una matriz $A \in \mathbb{R}^{n \times n}$ simétrica cuyos valores propios son $\lambda_i$ (por simplicidad, consideraremos que también son todos positivos) y sus vectores propios son $\mathbf{v}_i$, para $i \in \{1, 2, \dots, n\}$. Considere además que $\lambda_i > \lambda_j$, para todo $i < j$, es decir, los valores propios están ordenados. Considere ahora la matriz $A_1$ definida como:

$$
    A_1 = A - \lambda_1\mathbf{v}_1\,\mathbf{v}_1^T.
$$

**1. (5 puntos)** Demuestre que los vectores propios de $A$ también son los vectores propios de $A_1$. ¿Cuáles son los valores propios de $A_1$?

Respuesta: 
Como sabemos A es simetrica y ademas estan los valores propios ordenados lo que significa que tenemos el dominante 
primero en 1y todos positivos.
Primero tomamos el vector propio dominante de A, que multiplicamos por $A_1$,
$$
    A_1 \mathbf{v}_1 = (A - \lambda_1\mathbf{v}_1\,\mathbf{v}_1^T)\mathbf{v}_1.
$$
$$
    A_1 \mathbf{v}_1 = A\mathbf{v}_1 - \lambda_1\mathbf{v}_1\,\mathbf{v}_1^T\mathbf{v}_1.
$$
Sabiendo que $\mathbf{v}_1^T\mathbf{v}_1 = 1.$
$$
    A_1 \mathbf{v}_1 = 0.
$$
Por lo que v_1 es un vector propio de $A_1$ el cual tiene como valor propio el 0. Nos falta el resto, enntonces tomamos otro vector de A pero debe ser distinto de $v_1$, $v \not = v_1$,
$$
    A_1 \mathbf{v} = (A - \lambda_1\mathbf{v}_1\,\mathbf{v}_1^T)\mathbf{v}.
$$
$$
    A_1 \mathbf{v} = A\mathbf{v} - \lambda_1\mathbf{v}_1\,\mathbf{v}_1^T\mathbf{v}.
$$
Sabiendo que $\mathbf{v}_1^T\mathbf{v} = 0.$
$$
    A_1 \mathbf{v} = A\mathbf{v}.
$$

Finalmente nos queda q el vector propio $v$ de $A_1$ que se asocia a los valores de A, es decir, mantiene los valores y vectores propios con excepción de $v_1$ que tiene valor propio 0. Pero el vector propio dominante cambia, en este caso seria para $A_1$ el segundo de $A$.

**2. (10 puntos)** El proceso anterior, denominado *Deflation*, puede repetirse iterativamente para construir un algoritmo iterativo que permita encontrar los valores y vectores propios en sucesión. El algoritmo, paso a paso, puede definirse como:

1. Se define $A_0 = A$, $\lambda_0 = 0$ y $\mathbf{v}_0 = \mathbf{0}$.
2. Para $i \in \{1, 2, \dots, n\}$
    1. Se define $A_i = A_{i-1} - \lambda_{i-1}\mathbf{v}_{i-1}\,\mathbf{v}_{i-1}^T$.
    2. Se encuentra el valor y vector propio dominante de $A_{i}$, denotados por $\lambda_{i}$ y $\mathbf{v}_{i}$, respectivamente.
3. Se retorna el conjunto $\{\lambda_1, \lambda_2, \dots, \lambda_n\}$ y $\{ \mathbf{v}_{1}, \mathbf{v}_{2}, \dots, \mathbf{v}_{n}\}$.

Implemente la función `deflation`, el cual aplicará el algoritmo *Deflation* mediante el uso de *Power Iteration*. Esta función debe recibir como parámetros:
* La matriz $A$.
* Un *initial guess* $\mathbf{x}_0$.
* Un número de iteraciones.

La función debe retornar una lista con los valores propios encontrados y una lista de los vectores propios encontrados.

In [3]:
n=12
ev = 2/np.arange(1,n+1)
np.random.seed(0)
B = np.random.rand(n,n)
#print(B)
Q, R = np.linalg.qr(B)
#print(Q)
#print(R)
A = np.dot(np.dot(Q.transpose(), np.diag(ev)),Q) #Q^-1 * ev * Q
#print(A)

#Power itaracion de jupyter notebook del curso
def power_iteration(A, x, k, verbose=False):
    """
    Program 12.1 Power iteration
    Computes dominant eigenvector of square matrix
    Input: matrix A, initial (nonzero) vector x, number of steps k
    Output: dominant eigenvalue lam, eigenvector u
    """
    if verbose: print("Power Iteration Method\n%s"%('='*80))
    for j in range(k):
        u = x/np.linalg.norm(x)
        x = np.dot(A, u)
        lam = np.dot(u, x) #not really necessary to compute it at each iteration
        if verbose: print("k=%d, lambda=%+.3f, u=%s"%(j,lam,str(u.T)))
    u = x/np.linalg.norm(x)
    if verbose: print("k=%d, lambda=%+.3f, u=%s\n"%(j+1,lam,str(u.T)))
    return (lam, u)  

In [4]:
'''
Compute all the eigenvalues and eigenvectors using Deflation procedure.
Input:
A - (n x n matrix) Matrix to compute its eigenvalues and eigenvectors
x0 - (array) Initial guess for Power Iteration
it - (integer) Number of iteration for Power Iteration.
Output:
eig_values - (list) A list with the eigenvalues found by the algorithm
eig_vectors - (list) A list with the eigenvectors found by the algorithm
'''
def deflation(A, x0, it):
    k = it
    eig_values = []
    eig_vectors = []
    for  i in range(0,it):
        #print("Iteracion #", i)
        lam, u = power_iteration(A, x0, k, verbose=False)
        eig_values.append(lam)
        eig_vectors.append(u)
        A = A - np.outer(np.dot(lam,u),u.transpose())
    return eig_values, eig_vectors

In [5]:
it = 4
x0 = np.ones(len(A))
eig_values , eig_vectors = deflation(A,x0 ,it)
print(eig_values)
print(eig_vectors)

[1.983477417152032, 0.847023069920312, 0.7506289003447759, 0.3467499708439723]
[array([-0.28209586, -0.17040548, -0.0284793 ,  0.01265423,  0.08288897,
       -0.08997173,  0.23520642,  0.65603569, -0.03234983, -0.08583302,
        0.5615199 , -0.25695115]), array([ 3.59029953e-01,  5.46225340e-01, -4.14383094e-01,  1.99796278e-02,
       -8.25514091e-05,  3.26174044e-01,  3.03382201e-01, -5.93823583e-02,
       -3.23502211e-01, -1.36527099e-01,  2.29911469e-01,  1.50045616e-01]), array([-0.11381304,  0.07331188,  0.44597933, -0.63024044,  0.05292362,
       -0.16894893,  0.20404775,  0.01663173,  0.15143423,  0.36385857,
        0.09098227,  0.38563128]), array([ 0.30805751, -0.12602325,  0.44822931,  0.66863346, -0.13251052,
        0.18133157, -0.07731724,  0.19814061,  0.07880408,  0.01248307,
        0.13643158,  0.34724802])]


Para probar su algoritmo, utilice la función `generate_matrix` que construye una matriz simétrica a partir de una lista de valores propios específicos.

In [6]:
'''
Given a list of n real vaues, returns a symmetric matrix whose eigenvalues are the given values.
Input:
eig_values - (list) list of n real values that will be used as eigenvalues
Output: 
A - (n x n matrix) A symmetric matrix whose eigenvalues equals the values in eig_values list
Q - (n x n matrix) An orthonormal matrix whose columns are the eigenvectors
eig_values (list) A list with eigenvalues of A. It is the same list given as input.
'''
def generate_matrix(eig_values):
    n = len(eig_values)
    Q, _ = np.linalg.qr(np.random.rand(n, n))
    L = np.diag(eig_values)
    A = np.dot(Q, np.dot(L, Q.T))
    return A, Q, eig_values


A2, Q, eig_values = generate_matrix(eig_values)
print(A2)
print(Q)
print(eig_values)

[[ 1.07548328  0.52174915 -0.03599488  0.45237825]
 [ 0.52174915  0.93385553  0.25286136  0.35669185]
 [-0.03599488  0.25286136  0.71593696  0.06736024]
 [ 0.45237825  0.35669185  0.06736024  1.2026036 ]]
[[-0.59461576  0.17362389  0.57806141 -0.53116085]
 [-0.52679521 -0.47732322  0.21638808  0.66919769]
 [-0.11984815 -0.76172908 -0.3703767  -0.51790571]
 [-0.59544549  0.40222642 -0.69414862  0.0426177 ]]
[1.983477417152032, 0.847023069920312, 0.7506289003447759, 0.3467499708439723]


Genere matrices con los siguientes valores propios:
* $\{1, 2, \dots, 10\}$.
* $\{2, 2^2, 2^3, \dots, 2^{12}\}$.

Utilice las matrices generadas para probar su algoritmo. Su tarea será probada con otras matrices, pero puede utilizar estas para verificar la correctitud de su algoritmo.

In [7]:
#Matrices con valores propios {1,2....10}
eig_values_1 = [1,2,3,4,5,6,7,8,9,10]
A_1, Q, eig_values = generate_matrix(eig_values_1)
print(A_1)
print(Q)
print(eig_values_1)

[[ 5.79324671 -0.17400742 -0.90114668 -1.35357138  0.76104211 -1.7022969
  -0.15348536 -0.3038561  -0.57580074 -1.25186367]
 [-0.17400742  5.72227524 -0.36321186 -0.19449342 -0.07137248 -0.6033318
  -0.3537302   0.64283577 -1.94357181  0.4390138 ]
 [-0.90114668 -0.36321186  5.70692139  0.72007207 -0.52182538  1.12512462
  -0.47323057 -1.66984466 -0.00807752 -1.31349459]
 [-1.35357138 -0.19449342  0.72007207  5.29752895  0.77354893  0.12146796
  -0.99413745  0.07709354 -0.444168   -0.44056365]
 [ 0.76104211 -0.07137248 -0.52182538  0.77354893  6.44979073 -0.46716364
  -1.8066531  -0.20975758 -0.12715903 -1.8356868 ]
 [-1.7022969  -0.6033318   1.12512462  0.12146796 -0.46716364  5.01505097
  -0.68763342 -1.58713102 -2.15354446  0.37460313]
 [-0.15348536 -0.3537302  -0.47323057 -0.99413745 -1.8066531  -0.68763342
   3.90048432  1.07687474  0.01434994 -0.28328053]
 [-0.3038561   0.64283577 -1.66984466  0.07709354 -0.20975758 -1.58713102
   1.07687474  6.81117232 -0.24948842 -0.50341759]
 [

In [8]:
#Matrices con valores propios {2,2^2....2^10}
eig_values_2 = [2**1, 2**2 , 2**3 , 2**4 , 2**5 , 2**6 , 2**7 , 2*8 , 2*9 , 2**10,2**11,2**12]
A_2, Q, eig_values = generate_matrix(eig_values_2)
print(A_2)
print(Q)
print(eig_values_2)

[[ 244.22762006 -331.27065578  216.50135647 -243.57639403 -269.08552641
   294.85203279  351.8361167    -3.66578162 -330.70028707   96.31796206
    75.82063121  156.58497309]
 [-331.27065578  845.57849466  -21.87717788  -82.22538602  310.22635367
  -824.73032438 -336.18296678   54.93821967  398.03275566 -302.74738876
    14.29256276 -254.65625931]
 [ 216.50135647  -21.87717788 1026.14640555 -860.82136326 -434.16349079
    67.16811253  678.7859626   468.14571154 -787.0269554  -206.52009503
  -173.67828119  471.13101331]
 [-243.57639403  -82.22538602 -860.82136326 1012.81988658  499.35558795
   143.92787637 -756.00538302 -236.56827669  620.55194156  155.49290106
  -142.39301318 -283.30608805]
 [-269.08552641  310.22635367 -434.16349079  499.35558795  441.08085756
  -241.13399669 -534.88951437  -77.28148689  452.95893878  -72.12001223
  -120.19710151 -219.7352617 ]
 [ 294.85203279 -824.73032438   67.16811253  143.92787637 -241.13399669
  1006.2911962   324.79599799   59.94927218 -484.1618

**3. (15 puntos)** Implemente el algoritmo *Normalized Simultaneous Iteration* (NSI). Genere matrices de los siguientes tamaños, utilizando la función `generate_matrix`:

* $100 \times 100$.
* $1000 \times 1000$.
* $10000 \times 10000$.
* $15000 \times 15000$.

Estime teóricamente cuanta memoria requiere cada uno de los algoritmos. Además, ejecute ambos algoritmos sobre las matrices generadas y mida el tiempo de ejecución. Concluya respecto a los resultados.
##Conclusion##
Al ser la matrices tan grandes los tiempos de ejecucion se volvieron sumamente altos a tal nivel que para NSI no pude obtenerlos y el kernel se exploto. En ese sentido se debe a que este ultimo utiliza QR por lo que la descomposicion crea matrices de dimesiones muy altas y usa una distinta en cada iteracion, lo que vuelve una constante creacion de matrices logrando asi ocupar mucho espacio.

Respecto a la estimacion de cuanta memoria se requiere claramente notamos una gran diferencia estre sparses y no sparses, esto debido al tamaño de las matrices. Como se hace uso de las matrices varias veces y es necesario guardarlas para volver a utlizarlas, además de la descomposicion de estas, el tamaño se vuelve mucho más grande que el calculado, pero claramente el uso de NSI aumenta mucho el uso de memoria, llegando hasta los 2GB, donde un computador normal tiene 8GB, donde debemos descontar el S.O. lo que nos da un uso aprox de 5GB restantes. Por lo que podriamos almacenar en el mejor de los casos una matriz no sparse de 15000x15000 unas 3 veces y luego no tendremos mas memoria.

In [108]:
#Matrices No sparses
from scipy.stats import uniform
data = uniform.rvs(size=15000*15000, loc = 0, scale=2)
data = np.reshape(data, (15000, 15000))
data_size = data.nbytes/(1024**2)
print('Tamaño para 15000 '+ '%3.2f' %data_size + ' MB')
data = uniform.rvs(size=10000*10000, loc = 0, scale=2)
data = np.reshape(data, (10000, 10000))
data_size = data.nbytes/(1024**2)
print('Tamaño para 10000 '+ '%3.2f' %data_size + ' MB')
data = uniform.rvs(size=1000*1000, loc = 0, scale=2)
data = np.reshape(data, (1000, 1000))
data_size = data.nbytes/(1024**2)
print('Tamaño para 1000 '+ '%3.2f' %data_size + ' MB')
data = uniform.rvs(size=100*100, loc = 0, scale=2)
data = np.reshape(data, (100, 100))
data_size = data.nbytes/(1024**2)
print('Tamaño para 100 '+ '%3.2f' %data_size + ' MB')

Tamaño para 15000 1716.61 MB
Tamaño para 10000 762.94 MB
Tamaño para 1000 7.63 MB
Tamaño para 100 0.08 MB


In [110]:
#Matrices Sparce
data = uniform.rvs(size=100*100, loc = 0, scale=2)
data = np.reshape(data, (100, 100))
data_csr = sparse.csr_matrix(data)
data_csr_size = data_csr.data.size/(1024**2)
print('Tamaño para 100 Sparse '+ '%3.2f' %data_csr_size + ' MB')

data = uniform.rvs(size=1000*1000, loc = 0, scale=2)
data = np.reshape(data, (1000, 1000))
data_csr = sparse.csr_matrix(data)
data_csr_size = data_csr.data.size/(1024**2)
print('Tamaño para 1000 Sparse '+ '%3.2f' %data_csr_size + ' MB')

data = uniform.rvs(size=10000*10000, loc = 0, scale=2)
data = np.reshape(data, (10000, 10000))
data_csr = sparse.csr_matrix(data)
data_csr_size = data_csr.data.size/(1024**2)
print('Tamaño para 10000 Sparse '+ '%3.2f' %data_csr_size + ' MB')

data = uniform.rvs(size=15000*15000, loc = 0, scale=2)
data = np.reshape(data, (15000, 15000))
data_csr = sparse.csr_matrix(data)
data_csr_size = data_csr.data.size/(1024**2)
print('Tamaño para 15000 Sparse '+ '%3.2f' %data_csr_size + ' MB')



Tamaño para 100 Sparse0.01 MB
Tamaño para 1000 Sparse0.95 MB
Tamaño para 10000 Sparse95.37 MB
Tamaño para 15000 Sparse214.58 MB


In [9]:
def nsi(A, it):
    n = np.size(A[0])
    V = np.random.random(n*n).reshape(n,n)
    Q,R = np.linalg.qr(V)
    for i in range(it):
        Z = np.outer(A,Q)
        Q2,R2 = np.linalg.qr(Z)
    return Z

In [ ]:
#Generamos las matrices de los tamaños pedidos
eig_values_1 = np.random.random(100)
B_1, Q_1, eig_values_1 = generate_matrix(eig_values_1)
eig_values_2 = np.random.random(1000)
B_2, Q_2, eig_values_2 = generate_matrix(eig_values_2)
eig_values_3 = np.random.random(10000)
B_3, Q_3, eig_values_3 = generate_matrix(eig_values_3)
eig_values_4 = np.random.random(15000)
B_4, Q_4, eig_values_4 = generate_matrix(eig_values_4)

#B_1 = np.random.random([100,100])
#B_2 = np.random.random([1000,1000])
#B_3 = np.random.random([10000,10000])
#B_4 = np.random.random([15000,15000])

In [ ]:
#Comprobamos el tamaño de las matrices
print(B_1.shape,B_2.shape,B_3.shape,B_4.shape)

##Los tiempos fueron muy altos por lo que no pude comprobar las dimensiones##

In [260]:
#Para los tiempos de ejecucion para deflation
#Probando el algoritmo
it = 4
x0 = np.ones(len(B_1))
x1 = np.ones(len(B_2))
x2 = np.ones(len(B_3))
x3 = np.ones(len(B_4))


start_time = time()
eig_values_1 , eig_vectors_1 = deflation(B_1,x0 ,it)
elapsed_time = time() - start_time
print("----------Tiempo de ejecucion B_1-------\n",elapsed_time)
start_time = time()
eig_values_2 , eig_vectors_2 = deflation(B_2,x1 ,it)
elapsed_time = time() - start_time
print("----------Tiempo de ejecucion B_2-------\n",elapsed_time)
start_time = time()
eig_values_3 , eig_vectors_3 = deflation(B_3,x2 ,it)
elapsed_time = time() - start_time
print("----------Tiempo de ejecucion B_3-------\n",elapsed_time)
start_time = time()
eig_values_4 , eig_vectors_4 = deflation(B_4,x3 ,it)
elapsed_time = time() - start_time
print("----------Tiempo de ejecucion B_4-------\n",elapsed_time)

----------Tiempo de ejecucion B_1-------
 0.0021219253540039062
----------Tiempo de ejecucion B_2-------
 0.21138930320739746
----------Tiempo de ejecucion B_3-------
 26.49055004119873
----------Tiempo de ejecucion B_4-------
 501.620600938797


In [ ]:
#Para los tiempos de ejecucion para NSI
#Probando el algoritmo
it = 4

start_time = time()
nsi(B_1,it)
elapsed_time = time() - start_time
print("----------Tiempo de ejecucion B_1-------\n",elapsed_time)
start_time = time()
nsi(B_2,it)
elapsed_time = time() - start_time
print("----------Tiempo de ejecucion B_2-------\n",elapsed_time)
start_time = time()
nsi(B_3,it)
elapsed_time = time() - start_time
print("----------Tiempo de ejecucion B_3-------\n",elapsed_time)
start_time = time()
nsi(b_4,it)
elapsed_time = time() - start_time
print("----------Tiempo de ejecucion B_4-------\n",elapsed_time)
##La Memoria no dio y el kernel se reiniciaba##

----------Tiempo de ejecucion B_1-------
 326.5782608985901


## Sección 2: Uso de matrices *Sparse* (60 puntos)

Cuando una matriz es de tipo *Sparse*[1], es posible trabajar sobre matrices de mayor tamaño, debido a que se evita el almacenamiento de valores *innecesarios* (en este caso, los ceros).

**1. (5 puntos)** Implemente la función `read_sparse_matrix` que reciba la ruta a un archivo de texto plano que contiene una matriz *sparse*, y construya la matriz respectiva utilizando una estructura de datos especializada para matrices *sparse*[2]. El formato del archivo será:
* La primera línea del archivo contendrá un solo número `N`, donde `N` indica el tamaño de la matriz (matriz de tamaño `N`$\times$`N`).
* Cada una de las siguientes líneas contiene 3 números separados por espacios, que indicarán las entradas no nulas de la matriz. Si la línea contiene los números `i j a`, esto indica que en la fila `i` y columna `j`, la matriz tiene un coeficiente con valor `a`.

In [10]:
#Para implementar el algoritmo y aprovechar que esta es sparse usaremos la libreria de scipy.sparse
#leeremos el archivo que se nos da, de esta forma ahorramos espacio. 

In [11]:
'''
Read a file in the specified format to build a Sparse Matrix.
Input:
path - (string) A path to a file.
Output:
A - (N x N matrix) The matrix in a sparse format.
'''
def read_sparse_matrix(path):
    #Primero abrimos el archivo que contiene la matriz sparse
    file = open(path,"r")
    #La primera linea corresponde al tamaño de la matriz, por lo que usamos readline()
    n = int(file.readline())
    row , col , k = [],[],[]
    #Luego avanzamos en las lineas restantes que contienen la forma (row col k), es necesario separar cada uno para invocar 
    #la funcion csr_matriz la cual nos va formar la matriz sparse
    for i in file:
        auxr, auxc , auxk = i.split()
        row.append(float(auxr))
        col.append(float(auxc))
        k.append(float(auxk))
        #Ya separadas las posiciones y los valores realizamos la llamada a csr_matriz para formar la matriz Sparse
    return sparse.csc_matrix((k,(row,col)),shape=(n,n))

Cargue las matrices adjuntas en los archivos `matrix_10000.txt`, `matrix_20000.txt` y `matrix_50000.txt`, de dimensiones $10000 \times 10000$, $20000 \times 20000$ y $50000\times50000$ respectivamente, utlizando la función `read_sparse_matrix`. Considerando *double precision* para los números de punto flotante, indique cuánta memoria utilizarían estas matrices si fuesen cargadas utilizando arreglos bidimensionales o estructuras no-especializadas para matrices *sparse*. ¿Existe alguna diferencia al utilizar las estructuras de datos *sparse*?

Como se vio mas arriba en el calculo de memoria, existe una clara diferencia en el uso de memoria, por lo que utilizar matrices sparse disminuye la memoria usada. Sparce trabaja solo con datos que no son nulos, es decir, permite que se libere espacio, lo que ayuda a que no se almacenen datos innecesarios como lo hacen las matrices no-sparse.

In [12]:
path1 = "matrix_10000.txt"
path2 = "matrix_20000.txt"
path3 = "matrix_50000.txt"
MatrizSparceA = read_sparse_matrix(path1)
MatrizSparceB = read_sparse_matrix(path2)
MatrizSparceC = read_sparse_matrix(path3)

**2. (25 puntos)** Suponga ahora que la matriz simétrica $A$ en el algoritmo de *Deflation* es *Sparse*. Notar que luego de calcular el valor y vector propio dominante de $A$, el resultado obtenido para calcular $A_1$:

$$
    A_1 = A - \lambda_1\mathbf{v}_1\,\mathbf{v}_1^T,
$$

**no** necesariamente es una matriz *sparse*. Esto se debe a que la matriz $\mathbf{v}_1\mathbf{v}_1^T$ no necesariamente es *sparse* y al restarse con $A$, esta propiedad puede perderse.

A pesar de lo anterior, una propiedad importante de *Power Iteration* es que no opera direcamente sobre la matriz, si no que solo es requerida a través del producto **matriz-vector**. Notar que si se desea calcular $A_1\mathbf{u}$, donde $\mathbf{u}$ es un vector arbitrario, puede calcularse como

$$
    A_1\mathbf{u} 
    = \left(A - \lambda_1\mathbf{v}_1\,\mathbf{v}_1^T\right)\mathbf{u}
    = A\mathbf{u} - \lambda_1 \mathbf{v}_1 \left(\mathbf{v}_1^T\mathbf{u}\right).
$$

De esta manera, no es necesario construir explícitamente la matriz $\mathbf{v}_1\mathbf{v}_1^T$. De manera general, la matriz $A_i\mathbf{u}$ puede calcularse como

$$
    A_i\mathbf{u} = A\mathbf{u} - \sum_{k = 1}^{i}\lambda_k \left( \mathbf{v}_k^T\mathbf{u} \right) \mathbf{v}_k.
$$

Implemente la función `A_times_vector` que reciba como parámetros: 

* Una matriz de tipo *sparse*.
* Un entero $k$.
* Una lista con $k$ valores propios.
* Una lista con $k$ vectores propios.
* Un vector $\mathbf{u}$, arbitrario.

La función debe retornar el resultado numérico del producto matriz-vector $A_{k}\mathbf{u}$.

In [13]:
'''
Implementation of a Deflation matriz-vector product to avoid building non-sparse matrix
Input:
A - (N x N matrix) A matrix in a sparse format.
k - (integer) An integer.
k_eig_values - (list) A list with k eigenvalues of A.
k_eig_vectors - (list) A list with k eigenvectors of A.
u - (array) An arbitrary vector
Output:
b - (array) The result of a Deflation matrix times u, using the given eigenvalues and eigenvectors.
'''

#Usamos un vector arbitrario u
k=10000
u = np.random.random(k)

def A_times_vector(A, k, k_eig_values, k_eig_vectors, u):
    suma = 0
    #Recorremos los valores y vectores, esto son de tamaño k
    for i in range(k):
        #Extraemos uno por uno los valores y el vector correspondiente a k
        lam = k_eig_values[i]
        x = k_eig_vectors[i]
        #Lo primero que hacemos es multiplicar los valoresen parentesis de v^t * u
        multi1 = x*u
        #Para la sumatoria guardamos los valores de la iteracion anterior y multiplicamos el valor propio correspondiente
        #por el parentesis obtenido anteirormente donde este se multiplica por v.
        suma = suma + lam*(multi1*x)
        #Entonces para obtener b, multiplicamos A*u y le restamos la suma
        b = A*u - suma 
        #print(b)
    # b is the result of the matrix-vector product
    return b

In [ ]:
print(eig_values)
print(eig_vectors)
print(MatrizSparceA.shape)
A_times_vector(MatrizSparceA, k, eig_values, eig_vectors, u)

**3. (30 puntos)** Implemente la función `sparse_deflation`, modificando su implementación inicial de *Deflation*, que permita operar sobre matrices de gran dimenensión y que sean *sparse*. Para esto, utilice la implementación realizada de la función `A_times_vector` en combinación con el algoritmo de *Power Iteration*. La función `sparse_deflation` recibe como parámetros:
* La matriz $A$ de tipo *sparse*.
* El *initial guess* $\mathbf{x}_0$.
* El número de iteraciones a realizar en *Power Iteration*
* Un número $T$ que indique el número de valores y vectores propios que se desean obtener de la matriz $A$.

In [18]:
'''
Compute T eigenvalues and eigenvectors from the sparse matrix A, using Deflation.
Input:
A - (n x n matrix) Sparse matrix
x0 - (array) Initial guess for Power Iteration
it - (integer) Number of iterations for Power Iteration
T - (integer) Number of eigenvalues and eigenvectors to compute from A
Output:
eig_values - (list) List of T eigenvalues found by the algorithm
eig_vectors - (list) List of T eigenvectors found by the algorithm
'''
def power_iteration2(A, x, k,eig_values ,eig_vectors):
    for j in range(k):
        u = x/np.linalg.norm(x)
        x = A_times_vector(A, k, eig_values, eig_vectors, u) #np.dot(A, u)
        lam = np.dot(u, x)
    u = x/np.linalg.norm(x)
    return (lam, u)  

def sparse_deflation(A, x0, it, T):
    #Debemos usar la cantidad T de valores y vectores propios.
    eig_values = []
    eig_vectors = []
    for i in range(T):
        lam , u =  power_iteration2(A, x0, it,eig_values ,eig_vectors)
        eig_values.append(lam)
        eig_vectors.append(u)
        A = A - np.outer(np.dot(lam,u),u.transpose())
    return eig_values, eig_vectors

Aplique la función `sparse_deflation` sobre las matrices cargadas desde los archivos adjuntos, para distintos valores de $T$ y de iteraciones para *Power Iteration*. Mida los tiempos de ejecución y concluya al respecto. ¿Podría haber aplicado el algoritmo NSI para estas matrices? Justifique y concluya al respecto.


Respuesta: Cuando mayor es la cantidad de vectores y valores propios mayor es el tiempo de ejecucion, esto debido a que la matriz crece y forma una matriz más grande. Si bien la matriz es sparse, el numero de iteraciones aumenta la cantidad de matrices y se usan 3 algoritmos, lo que produce un tiempo de ejecucion mayor pero este es mas bajo que usar matrices no sparse, debido a que no se realizan calculos innecesarios y la matriz es mas "pequeña". Si aplicaramos NSI tendriamos que usar la descomposicion por lo que aumentaria el espacio de memoria necesario y como vimos anteriormente ya es alto. 

---
# Instrucciones:

* **Importante, Asegúrese de responder TODO lo que la pregunta pide.**
* La estructura de la tarea es la siguiente:
     1. Título, nombre de estudiante, email y rol.
     2. Responder cada pregunta de forma personal.
     5. Referencias. Es muy importante incluir todas las fuentes usadas, de otra forma se considera que lo no se ha citado adecuadamente es su trabajo.
* La tarea debe ser realizada en `Jupyter Notebook` (`Python3`) entregado.
* Recuerde responder la encuesta en el plazo establecido
* Se evaluará la correcta utilización de librerias `NumPy`, `SciPy`, `Matplotlib` y `ipywidgets`, entre otras, así como la **correcta implementación de algoritmos vectorizados**.
* **MUY IMPORTANTE** El archivo de entrega debe denominarse TareaN-rol.tar.gz y _notebook_ debe tener como nombre TareaN-rol.ipynb, donde $N$ es el número de la tarea y debe contener un directorio con todos los archivos necesarios para ejecutar el notebook, junto con un archivo README indicando explícitamente las librerías o módulos utilizados, nombre y rol del estudiante. Por cada error en este ambito implicará un descuento de 30 puntos.
* El descuento por día de atraso será de $30$ puntos, con un máximo de 1 día de atraso. No se recibirán entregas después de este día.
* Debe citar toda fuente de código externo. 
* El trabajo es personal, no se permite compartir código ni utilizar código de otros, aunque sí se sugiere discutir aspectos generales con sus compañeros.
* En caso de sospecha de no cumplimiento de estas instrucciones, se solicitará al involucrado o la involucrada a aclarar la situación. Dependiendo de la justificación se decidirá su calificación, la cual podrá o no ser penalizada.
* El no seguir estas instrucciones, implica descuentos en su nota obtenida.

---

# Referencias


[1] Sparse matrix, https://en.wikipedia.org/wiki/Sparse_matrix

[2] Sparse module, Scipy. https://docs.scipy.org/doc/scipy/reference/sparse.html
[3] Jupyter Notebook Curso. https://github.com/tclaudioe/Scientific-Computing 
[4] Iteración simultánea y algoritmo de QR. https://www.inf.utfsm.cl/~parce/cc2/clase6-PA.html
[5] Sperse Matrix. https://scipy.github.io/devdocs/sparse.html
[6] Memory Sparse Matrix. https://cmdlinetips.com/2018/03/sparse-matrices-in-python-with-scipy/